# Getting started with Langchain

In this tutorial, we will walk through the basics of using LangChain to create a simple AI chatbot that can answer questions using OpenAI and Serpapi.

In [1]:
# import libraries
import os
from langchain.llms import OpenAI

In [4]:
#create a new openai api key
#os.environ["OPENAI_API_KEY"] = "..."

In [5]:
# set up openai api key
openai_api_key = os.environ.get('OPENAI_API_KEY')

### The temperature of a llm is a hyperparameter that controls the randomness of the output. It is a value between 0 and 1. A higher temperature will result in more random output. A lower temperature will result in more predictable output. For this tutorial, we will set the temperature to 0.9. You can play around with this yourself to see how it affects the output.

In [6]:
# create a llm
llm = OpenAI(temperature = 0.9)

In [7]:
text = "What are 4 countries where they eat a lot of potatoes?"
print(llm(text))



1. Ireland 
2. Russia 
3. United States 
4. Germany


### This works! But what if we what if we want to ask a question about a different food? We can use the prompt template class to do this.

# Getting started with prompt templates

### A prompt template is a string that contains variables that can be filled in with different values. For example, you could have a prompt template that looks like this:

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 4 countries where they eat a lot of {food}?",
)

In [10]:
print(prompt.format(food="potatoes"))

What are 4 countries where they eat a lot of potatoes?


In [11]:
print(llm(prompt.format(food="potatoes")))



1. Ireland 
2. Germany 
3. Russia 
4. Poland


Notice how the answer is different from the previous one. This is because of the randomness of the llm. If we want to get the same answer every time, we can set the seed of the llm.

In [13]:
print(llm(prompt.format(food="rice")))



1. China 
2. India 
3. Indonesia 
4. Japan


# Chaining

### Here, we combine LLMs and prompts in multistep workflows using the prompt template class.

In [14]:
# import libraries
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [15]:
llm = OpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 4 countries where they eat a lot of {food}?",
)

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
print(chain.run("potatoes"))
print(chain.run("rice"))



1. Ireland
2. United States
3. Germany
4. United Kingdom


1. China
2. Japan
3. India
4. Indonesia


# Agents: Dynamically call chains based on user input

SerpApi is a search engine results page (SERP) scraping and parsing API that allows developers to retrieve and analyze data from various search engines, including Google, Bing, Yahoo, and more.


To use SerpApi in Python, you will need to install the SerpApi Python module using pip. An API key can be created here:

[Serpapi key](https://serpapi.com/users/sign_up)

[Serpapi documentation](https://python.langchain.com/en/latest/modules/agents/tools/examples/serpapi.html)

In the next part, we are going to combine OpenAI with SerpApi to create a chatbot that can answer questions with internet search results.

[Agent langchain documentation](https://python.langchain.com/en/latest/modules/agents.html)




In [89]:
# Install serpapi
#!pip install google-search-results

In [18]:
#import libraries
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [19]:
# Load the model
llm = OpenAI(temperature=0)

Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.
Here we use the SerpApi and llm-math tools. We will use the SerpApi tool to search the internet for answers to our questions. We will use the llm-math tool to answer math questions.

[Tools documentation](https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html)

In [2]:
# Load in some tools to use

#os.environ["SERPAPI_API_KEY"] = "..."

tools = load_tools(["serpapi", "llm-math"], llm=llm)

NameError: name 'load_tools' is not defined

### Finally, let's initialize an agent with:
 1. The tools
 2. The language model
 3. The type of agent we want to use.

In [21]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

For the agent, I have chosen the zero-shot-react-description agent. This agent uses the ReAct framework to determine which tool to use based solely on the tool’s description. Any number of tools can be provided. This agent requires that a description is provided for each tool.

[Agent types](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)

By setting verbose=True, we can see the agent's internal state as it processes the input.

Other agent types could also be chosen

Now, let us ask our agent a question for which it will use the Serpapi tool to search the internet for an answer and need to calculate the answer using the llm-math tool.

In [22]:
agent.run("What is the hight of Obama? And how many cans of coke can you stack to reach that height?")



> Entering new AgentExecutor chain...
 I need to find out Obama's height and then use a calculator to figure out how many cans of coke I need.
Action: Search
Action Input: Obama's height
Observation: 6′ 2″
Thought: I need to convert this to centimeters and then use a calculator to figure out how many cans of coke I need.
Action: Calculator
Action Input: 6 feet 2 inches in centimeters
Observation: Answer: 187.96
Thought: I now know the number of centimeters and can use a calculator to figure out how many cans of coke I need.
Action: Calculator
Action Input: 187.96 cm divided by 12 cm (the height of a can of coke)
Observation: Answer: 15.663333333333334
Thought: I now know the final answer
Final Answer: 15.66 cans of coke can be stacked to reach Obama's height of 6 feet 2 inches.

> Finished chain.


"15.66 cans of coke can be stacked to reach Obama's height of 6 feet 2 inches."

# Memory: Add state to chains and agents

By adding memory to our agents, we can make them more dynamic and interactive. We can have a conversation and the agent will remember what we said and use that information to answer our questions.


In [30]:
from langchain import OpenAI, ConversationChain

In [31]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

In [32]:
conversation.predict(input="Hi how are you doing!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi how are you doing!
AI:

> Finished chain.


" Hi there! I'm doing great, thanks for asking. How about you?"

In [33]:
conversation.predict(input="I'm doing well! Just having a conversation with my newly created langchain agent with memory.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi how are you doing!
AI:  Hi there! I'm doing great, thanks for asking. How about you?
Human: I'm doing well! Just having a conversation with my newly created langchain agent with memory.
AI:

> Finished chain.


' That sounds really cool! What kind of conversations have you been having?'

In [34]:
conversation.predict(input="I would like to get to know a bit about what LangChain is")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi how are you doing!
AI:  Hi there! I'm doing great, thanks for asking. How about you?
Human: I'm doing well! Just having a conversation with my newly created langchain agent with memory.
AI:  That sounds really cool! What kind of conversations have you been having?
Human: I would like to get to know a bit about what LangChain is
AI:

> Finished chain.


' Sure! LangChain is a natural language processing platform that enables developers to create AI-powered chatbots and virtual assistants. It provides a suite of tools to help developers build, train, and deploy their bots quickly and easily. It also offers a range of features such as natural language understanding, sentiment analysis, and text-to-speech capabilities.'

# Continue the conversation yourself!

In [ ]:
conversation.predict(input="...")

# This is the end of the tutorial. We hope you enjoyed it!

Follow me on Github and Medium for more content:

- [Github](https://github.com/rubentak)
- [Medium](https://medium.com/@rubentak)